In [1]:
import os
import numpy as np
import sys
from sklearn import svm

In [2]:
script_dir=os.getcwd()
rel_path="Input_data/UCI_HAR/"
abs_path=os.path.join(script_dir,rel_path)

In [3]:
X_train=np.loadtxt(abs_path+'train/X_train.txt')

In [4]:
X_train.shape

(7352, 561)

In [5]:
y_train=np.loadtxt(abs_path+'train/y_train.txt')

In [6]:
y_train.shape

(7352,)

In [7]:
clf=svm.LinearSVC(dual=False)
clf.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [8]:
X_test=np.loadtxt(abs_path+'test/X_test.txt')

In [9]:
y_test=np.loadtxt(abs_path+'test/y_test.txt')

In [10]:
y_test.shape

(2947,)

In [11]:
test_out=clf.predict(X_test)
test_success=(2947.0-np.count_nonzero(test_out-y_test))/2947.0

In [12]:
test_success

0.9664065151001018

### Trying to predict subject, not accurate at all!

subject_train=np.loadtxt(abs_path+'train/subject_train.txt')

subject_test=np.loadtxt(abs_path+'test/subject_test.txt')

subject_test.shape

clf_subject=svm.LinearSVC(dual=False)
clf_subject.fit(X_train,subject_train)

test_subject_out=clf_subject.predict(X_test)

test_subject_success=(2947.0-np.count_nonzero(test_subject_out-subject_test))/2947.0

test_success

test_subject_out

subject_test

In [48]:
adv_examples_test=np.zeros((2947,561))

count_pure_adv=0.0
count_adv=0.0
count_wrong=0.0
print_flag=0
count_correct_2=0.0
for i in range(2947):
    x_ini=(X_test[i,:]).reshape((1,561))
    ini_class=clf.predict(x_ini)
    x_adv=(x_ini-2.5*(clf.coef_[ini_class[0]-1,:]/(np.linalg.norm(clf.coef_[ini_class[0]-1,:])))).reshape((1,561))
    adv_examples_test[i,:]=x_adv
    final_class=clf.predict(x_adv)
    if y_test[i]==ini_class[0]:
        count_correct_2=count_correct_2+1
    if ini_class[0]!=final_class[0]:
        count_adv=count_adv+1
    if y_test[i]!=final_class[0]:
        count_wrong=count_wrong+1
    if y_test[i]!=final_class[0] and ini_class[0]==y_test[i]:
        count_pure_adv=count_pure_adv+1
#     if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
#         #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         print_flag=print_flag+1
print count_wrong/2947.0, count_adv/2947.0, count_pure_adv/count_correct_2

/Users/arjunbhagoji/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.964031218188 0.996267390567 0.996137640449


In [14]:
adv_examples_train=np.zeros((7352,561))

count_pure_adv=0.0
count_adv=0.0
count_wrong=0.0
print_flag=0
for i in range(7352):
    x_ini=(X_train[i,:]).reshape((1,561))
    ini_class=clf.predict(x_ini)
    x_adv=(x_ini-1.0*(clf.coef_[ini_class[0]-1,:]/(np.linalg.norm(clf.coef_[ini_class[0]-1,:])))).reshape((1,561))
    adv_examples_train[i,:]=x_adv
    final_class=clf.predict(x_adv)
    if ini_class[0]!=final_class[0]:
        count_adv=count_adv+1
    if y_train[i]!=final_class[0]:
        count_wrong=count_wrong+1
    if y_train[i]!=final_class[0] and ini_class[0]==y_train[i]:
        count_pure_adv=count_pure_adv+1
#     if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
#         #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         print_flag=print_flag+1
print count_wrong/7352.0, count_adv/7352.0, count_pure_adv/7352.0

/Users/arjunbhagoji/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.787948857454 0.792709466812 0.787948857454


In [15]:
ini_class

array([ 2.])

In [16]:
final_class

array([ 3.])

In [17]:
for i in range(6):
    print np.linalg.norm(clf.coef_[i,:])

4.17714260686
6.08051615939
4.03492342873
10.588368732
10.1657873574
1.71910641807


In [18]:
np.linalg.norm((x_adv-x_ini))

0.99999999999999989

In [19]:
np.sqrt(np.sum((x_adv-x_ini)**2)/561.0)

0.0422200330920749

In [20]:
np.where(x_adv-x_ini>0.1)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([ 41,  50,  89,  92, 117, 156, 197, 198, 381]))

In [21]:
np.mean(x_ini)

-0.40533133098413543

In [22]:
np.mean(x_adv)

-0.40519730319940994

In [23]:
np.std(x_ini)

0.48007378511187287

In [24]:
np.linalg.norm(x_ini)/np.sqrt(561)

0.62830273517549762

### Success with PCA

In [34]:
from sklearn.decomposition import PCA

rd=30

PCA_in_train=X_train
PCA_in_test=X_test

### Doing PCA over the training data
#Fitting the PCA model on training data
pca=PCA(n_components=30)
pca.fit(PCA_in_train)
#Transforming the training, validation and test data
X_train_dr=pca.transform(PCA_in_train)
X_test_dr=pca.transform(PCA_in_test)
X_adv_dr=pca.transform(adv_examples_test)

clf_pca=svm.LinearSVC(dual=False)

clf_pca.fit(X_train_dr,y_train)

test_out_pca=clf_pca.predict(X_test_dr)
test_success_pca=(2947.0-np.count_nonzero(test_out_pca-y_test))/2947.0

In [35]:
test_success_pca

0.9124533423820834

In [36]:
X_train_dr.shape

(7352, 30)

In [37]:
X_test_recons=pca.inverse_transform(X_test_dr)

In [38]:
test_out_recons=clf.predict(X_test_recons)
test_success_recons=(2947.0-np.count_nonzero(test_out_recons-y_test))/2947.0

In [39]:
test_success_recons

0.8978622327790974

### Testing reconstruction based defense

In [40]:
rev_adv=pca.inverse_transform(X_adv_dr)

In [50]:
count_pure_adv_2=0.0
count_adv_2=0.0
count_wrong_2=0.0
count_mis=0.0
count_correct_2=0.0
for i in range(2947):
    x_ini_2=(X_test[i,:]).reshape((1,561))
    ini_class_2=clf.predict(x_ini_2)
    #x_adv=(x_ini-1.0*(clf.coef_[ini_class[0],:]/(np.linalg.norm(clf.coef_[ini_class[0],:])))).reshape((1,784))
    x_adv_2=rev_adv[i,:].reshape((1,561))
    final_class_2=clf.predict(x_adv_2)
    if y_test[i]==ini_class_2[0]:
        count_correct_2=count_correct_2+1
    if ini_class_2[0]!=final_class_2[0]:
        count_adv_2=count_adv_2+1
    if y_test[i]!=final_class_2[0]:
        count_wrong_2=count_wrong_2+1
    if y_test[i]!=final_class_2[0] and ini_class_2[0]==y_test[i]:
        count_pure_adv_2=count_pure_adv_2+1
#     if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
#         #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         print_flag=print_flag+1
print count_wrong_2/2947.0, count_adv_2/2947.0, count_pure_adv_2/count_correct_2

0.232100441127 0.24737020699 0.227528089888


In [52]:
count_pure_adv_2=0.0
count_adv_2=0.0
count_wrong_2=0.0
count_mis=0.0
count_correct_2=0.0
for i in range(2947):
    x_ini_2=(X_test_dr[i,:]).reshape((1,rd))
    ini_class_2=clf_pca.predict(x_ini_2)
    x_adv_2=(x_ini_2-1.0*(clf_pca.coef_[ini_class[0]-1,:]/(np.linalg.norm(clf_pca.coef_[ini_class[0]-1,:])))).reshape((1,rd))
    #x_adv_2=rev_adv[i,:].reshape((1,561))
    final_class_2=clf_pca.predict(x_adv_2)
    if y_test[i]==ini_class_2[0]:
        count_correct_2=count_correct_2+1
    if ini_class_2[0]!=final_class_2[0]:
        count_adv_2=count_adv_2+1
    if y_test[i]!=final_class_2[0]:
        count_wrong_2=count_wrong_2+1
    if y_test[i]!=final_class_2[0] and ini_class_2[0]==y_test[i]:
        count_pure_adv_2=count_pure_adv_2+1
#     if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
#         #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         print_flag=print_flag+1
print count_wrong_2/2947.0, count_adv_2/2947.0, count_pure_adv_2/count_correct_2

/Users/arjunbhagoji/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0.163556158806 0.128605361384 0.108962439569


In [ ]:
count_pure_adv_2=0.0
count_adv_2=0.0
count_wrong_2=0.0
count_mis=0.0
for i in range(2947):
    x_ini_2=(X_test_dr[i,:]).reshape((1,rd))
    ini_class_2=clf_pca.predict(x_ini_2)
    #x_adv_2=(x_ini_2-1.0*(clf_pca.coef_[ini_class[0]-1,:]/(np.linalg.norm(clf_pca.coef_[ini_class[0]-1,:])))).reshape((1,rd))
    x_adv_2=X_adv_dr[i,:].reshape((1,rd))
    final_class_2=clf_pca.predict(x_adv_2)
    if ini_class_2[0]!=final_class_2[0]:
        count_adv_2=count_adv_2+1
    if y_test[i]!=final_class_2[0]:
        count_wrong_2=count_wrong_2+1
    if y_test[i]!=final_class_2[0] and ini_class_2[0]==y_test[i]:
        count_pure_adv_2=count_pure_adv_2+1
#     if y_train[i]!=final_class[0] and ini_class[0]==y_train[i] and print_flag==0:
#         #plt.imshow((x_ini.reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         plt.imshow(((x_adv-x_ini).reshape((28,28)))*255, cmap='gray', interpolation='nearest', vmin=0, vmax=255)
#         print_flag=print_flag+1
print count_wrong_2/2947.0, count_adv_2/2947.0, count_pure_adv_2/2947.0